# Preprocessing
in this notebook we developed the code to download images that will be used during the course of the project

### Imported libraries

In [ ]:
# Downloads
import os
import requests

# Images and Data
from PIL import Image

#!pip install opencv-python
import cv2

import pandas as pd
import numpy as np

# for dividing images
from math import floor
import random


2024-04-28 12:01:55.461877: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


only run if using colab

In [ ]:

#from google.colab import drive
#drive.flush_and_unmount()
#drive.mount("/content/drive")

### Paths to diferent directories
it's necessary to change the paths to the right directories where the files are saved

In [ ]:
dataset = "/Users/macbook/Documents/DL_Project/fitzpatrick17k.csv"

## Data preparation/cleaning

We start by loading and inspecting the data from "fitzpatrick17k.csv"

In [ ]:
dataset = pd.read_csv(dataset)


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16577 entries, 0 to 16576
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   md5hash                16577 non-null  object
 1   fitzpatrick_scale      16577 non-null  int64 
 2   fitzpatrick_centaur    16577 non-null  int64 
 3   label                  16577 non-null  object
 4   nine_partition_label   16577 non-null  object
 5   three_partition_label  16577 non-null  object
 6   qc                     504 non-null    object
 7   url                    16536 non-null  object
 8   url_alphanum           16577 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.1+ MB


In [ ]:
dataset.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,url,url_alphanum
0,5e82a45bc5d78bd24ae9202d194423f8,3,3,drug induced pigmentary changes,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmminoc...
1,fa2911a9b13b6f8af79cb700937cc14f,1,1,photodermatoses,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicpphoto...
2,d2bac3c9e4499032ca8e9b07c7d3bc40,2,3,dermatofibroma,benign dermal,benign,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicdderma...
3,0a94359e7eaacd7178e06b2823777789,1,1,psoriasis,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...
4,a39ec3b1f22c08a421fa20535e037bba,1,1,psoriasis,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...


check different values from relevant columns

In [ ]:
dataset['qc'].unique()

array([nan, '1 Diagnostic', '5 Potentially', '3 Wrongly labelled',
       '2 Characteristic', '4 Other'], dtype=object)

doesn't make sence to keep Wrongly labelled images so we drop this instances

In [ ]:
dataset = dataset.loc[dataset['qc'] != '3 Wrongly labelled']

Check inconsistencies on URL links

In [ ]:
dataset['url'].isnull().sum()

41

To many null we drop this instances

In [ ]:
dataset = dataset[dataset['url'].notnull()] 

save the 'clean' data set so it can be used in other notebooks

In [ ]:
dataset.to_csv('dataset_clean')

## Downloading images

We created a fuctions to download images with the Url from the dataset column

In [ ]:
def download_image(url, label, main_filepath):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'Accept': 'image/webp,image/apng,image/*,*/*;q=0.8'
    }
    filename = url.split('/')[-1]
    if not '.' in filename:
        filename += ".jpg"
    label_dir = os.path.join(main_filepath, label)
    os.makedirs(label_dir, exist_ok=True)
    file_path = os.path.join(label_dir, filename)

    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 404:
            print(f"Error 404: URL not found {url}")
            return
        elif response.status_code == 406:
            print(f"Error 406: Not Acceptable - content cannot be served according to the Accept headers {url}")
            return
        elif response.status_code != 200:
            print(f"Error {response.status_code}: Unable to download image {url}")
            return

        if 'image' in response.headers.get('Content-Type', ''):
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {file_path}")
        else:
            print(f"Skipping non-image URL: {url}")
    except requests.RequestException as e:
        print(f"Error downloading {filename}: {e}")

def download_images(df, main_filepath):
    for index, row in df.iterrows():
        download_image(row['url'], row['label'],main_filepath)

download images and save them on specific directory 

In [ ]:
main_filepath = "/content/drive/My Drive/Fac/DL2/main_filepath"
download_images(dataset, main_filepath)

# Distributing Images:

function to create the directories where we will store the diferetn set of images, from train, val and test

In [ ]:
def create_dataset_directories(main_directory):
    directories = ['training', 'validation', 'testing']

    for directory in directories:
        dir_path = os.path.join(main_directory, directory)
        os.makedirs(dir_path, exist_ok=True)
        print(f"Created directory: {dir_path}")

folders_filepath = "/content/drive/My Drive/Fac_Adri"
create_dataset_directories(folders_filepath)

Created directory: /content/drive/My Drive/Fac_Adri/training
Created directory: /content/drive/My Drive/Fac_Adri/validation
Created directory: /content/drive/My Drive/Fac_Adri/testing


We also made a function to divide the images throght the dir we created
- first we check if the function will count and divide stratrified the diferent labels
- then we confirm how many images were saved on main_filepath

In [ ]:
grouped = dataset.groupby('label')
label_counts = grouped.size()

train_counts = {}
val_counts = {}
test_counts = {}

for label, group_df in grouped:
    num_images = len(group_df)
    num_train = int(num_images * 0.75)
    num_val = int(num_images * 0.15)
    num_test = num_images - num_train - num_val

    train_counts[label] = num_train
    val_counts[label] = num_val
    test_counts[label] = num_test

print("Train counts:", train_counts)
print("Validation counts:", val_counts)
print("Test counts:", test_counts)

Train counts: {'acanthosis nigricans': 69, 'acne': 137, 'acne vulgaris': 251, 'acquired autoimmune bullous diseaseherpes gestationis': 50, 'acrodermatitis enteropathica': 69, 'actinic keratosis': 131, 'allergic contact dermatitis': 322, 'aplasia cutis': 54, 'basal cell carcinoma': 351, 'basal cell carcinoma morpheiform': 46, 'becker nevus': 47, 'behcets disease': 47, 'calcinosis cutis': 60, 'cheilitis': 79, 'congenital nevus': 51, 'dariers disease': 114, 'dermatofibroma': 59, 'dermatomyositis': 113, 'disseminated actinic porokeratosis': 45, 'drug eruption': 150, 'drug induced pigmentary changes': 53, 'dyshidrotic eczema': 62, 'eczema': 153, 'ehlers danlos syndrome': 95, 'epidermal nevus': 68, 'epidermolysis bullosa': 52, 'erythema annulare centrifigum': 66, 'erythema elevatum diutinum': 41, 'erythema multiforme': 177, 'erythema nodosum': 60, 'factitial dermatitis': 49, 'fixed eruptions': 93, 'folliculitis': 256, 'fordyce spots': 75, 'granuloma annulare': 158, 'granuloma pyogenic': 56, 

In [ ]:
original_dataset_dir = '/content/drive/My Drive/Fac_Adri/main_filepath'
labels = [d for d in os.listdir(original_dataset_dir) if os.path.isdir(os.path.join(original_dataset_dir, d))]
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
num_files = 0

for label in labels:
    label_dir = os.path.join(original_dataset_dir, label)
    files = [f for f in os.listdir(label_dir) if os.path.isfile(os.path.join(label_dir, f)) and os.path.splitext(f)[1].lower() in image_extensions]
    num_files += len(files)

print(num_files)

16517


the paths with the dirs we createad before

In [ ]:
base_dir = '/content/drive/My Drive/Fac/DL2/main_filepath'
train_dir = '/content/drive/My Drive/Fac/DL2/training'
val_dir = '/content/drive/My Drive/Fac/DL2/validation'
test_dir = '/content/drive/My Drive/Fac/DL2/testing'

code to distribute images
- it will go to the base_dir and divide the images base on the counts we made before
- before the division it suffles the images so its divided at random
- finally it saves a copy to the new dir we created 

In [ ]:
labels = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

def copy_files(filenames, source_dir, dest_dir):
    for fname in filenames:
        src = os.path.join(source_dir, fname)
        dst = os.path.join(dest_dir, fname)
        shutil.copyfile(src, dst)

for label in labels:
    label_dir = os.path.join(base_dir, label)
    files = [f for f in os.listdir(label_dir) if os.path.isfile(os.path.join(label_dir, f)) and os.path.splitext(f)[1].lower() in image_extensions]

    num_files = len(files)
    num_train = floor(num_files * 0.75)
    num_val = floor(num_files * 0.15)
    num_test = num_files - (num_train + num_val)

    shuffled_files = random.sample(files, len(files))

    train_files = shuffled_files[:num_train]
    val_files = shuffled_files[num_train:num_train + num_val]
    test_files = shuffled_files[num_train + num_val:]

    train_label_dir = os.path.join(train_dir, label)
    val_label_dir = os.path.join(val_dir, label)
    test_label_dir = os.path.join(test_dir, label)

    os.makedirs(train_label_dir, exist_ok=True)
    os.makedirs(val_label_dir, exist_ok=True)
    os.makedirs(test_label_dir, exist_ok=True)

    copy_files(train_files, label_dir, train_label_dir)
    copy_files(val_files, label_dir, val_label_dir)
    copy_files(test_files, label_dir, test_label_dir)


Finaly we print the counts to check consistency with number of images

In [ ]:
labels = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

num_train_total = 0
num_val_total = 0
num_test_toal = 0

for label in labels:
    label_dir = os.path.join(base_dir, label)
    files = [f for f in os.listdir(label_dir) if os.path.isfile(os.path.join(label_dir, f)) and os.path.splitext(f)[1].lower() in image_extensions]

    num_files = len(files)
    num_train = floor(num_files * 0.75)
    num_train_total += num_train
    num_val = floor(num_files * 0.15)
    num_val_total += num_val
    num_test = num_files - (num_train + num_val)
    num_test_toal += num_test

print(num_train_total)
print(num_val_total)
print(num_test_toal)


12340
2419
1750


## Sizes of images:

we started to create a simple function that will get the size of a provided image

In [ ]:
def get_file_size(file_path):
    return os.path.getsize(file_path)

image_sizes = {}

image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

for label in os.listdir(train_dir):
    label_dir = os.path.join(train_dir, label)
    if os.path.isdir(label_dir):
        label_image_sizes = []
        for file_name in os.listdir(label_dir):
            if any(file_name.lower().endswith(ext) for ext in image_extensions):
                file_path = os.path.join(label_dir, file_name)
                label_image_sizes.append(get_file_size(file_path))
        image_sizes[label] = label_image_sizes

for label, sizes in image_sizes.items():
    print(f"Label: {label}")
    for size in sizes:
        print(f"Image Size: {size} bytes")
    print("\n")

then we have another fuction to look inside the main_directory and look for the smallest image

In [ ]:
def find_smallest_images(main_directory):
    smallest_image = None
    smallest_size = float('inf')

    for label_dir in os.listdir(main_directory):
        label_dir_path = os.path.join(main_directory, label_dir)

        if os.path.isdir(label_dir_path):
          for filename in os.listdir(label_dir_path):
                  filepath = os.path.join(label_dir_path, filename)

                  with Image.open(filepath) as img:
                      width, height = img.size
                      image_size = width * height
                      if image_size < smallest_size:
                          smallest_size = image_size
                          smallest_image = filepath

    return smallest_size

train_dir = '/content/drive/My Drive/Fac/DL/training'
smallest_image = find_smallest_images(train_dir)
print("Smallest image:", smallest_image)

Smallest image: /content/drive/My Drive/Fac/DL/training/mucinosis/follicular-mucinosis41.jpg


we mada some ajustmentsto the function so it could find the 5% smalest images which heled us improve the results of our models

In [ ]:
def find_smallest_images(main_directory):
    widths = []
    heights = []

    for label_dir in os.listdir(main_directory):
        label_dir_path = os.path.join(main_directory, label_dir)

        if os.path.isdir(label_dir_path):
            for filename in os.listdir(label_dir_path):
                filepath = os.path.join(label_dir_path, filename)
                with Image.open(filepath) as img:
                    width, height = img.size
                    widths.append(width)
                    heights.append(height)

    widths = np.array(widths)
    heights = np.array(heights)

    sorted_widths = np.sort(widths)
    sorted_heights = np.sort(heights)

    lowest_5_percent = int(0.05 * len(sorted_widths))
    width_in_lowest_5_percent = sorted_widths[lowest_5_percent]
    height_in_lowest_5_percent = sorted_heights[lowest_5_percent]

    return width_in_lowest_5_percent, height_in_lowest_5_percent

train_dir = 'C:/Users/User/Documents/Fac/training'
smallest_width, smallest_height = find_smallest_images(train_dir)
print("Width of the image in the lowest 5%:", smallest_width)
print("Height of the image in the lowest 5%:", smallest_height)


Width of the image in the lowest 25%: 290
Height of the image in the lowest 25%: 192
